In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import bs4
import numpy as np

In [19]:
# Set up selenium to use Firefox
options = Options()
options.headless = True #No need to open a browser window

# Example of manaully specifying the WebDriver's location: 
driver = webdriver.Firefox(executable_path="../Others/geckodriver.exe",options=options)

In [20]:
def table(url, date, match = '1'): # default set match = 1 for checking if it is a race date only
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 10 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(((By.XPATH, '//table[@class="table_bd f_tac race_table"]'))))
    except Exception as e: 
        print(e, date, match)
        return []
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    table = soup.find('table', class_ = 'table_bd f_tac race_table')

    output_list = [] 
    
    for tr in table.find('tbody').find_all('tr'):
        cols = []
        position = [td.string for td in tr('td')]
        section_time = [p.string for p in tr('p')]
        margin_behind = [i.string for i in tr('i')]
        
        # using filter()
        # to remove None values in list
        position = list(filter(None, position))
        section_time = list(filter(None, section_time))
        margin_behind = list(filter(None, margin_behind))
        
        # keep it length of 6 total
        section_time = section_time + [''] * (6 - len(section_time))
        margin_behind = margin_behind + [''] * (6 - len(margin_behind))
        
        
        cols.extend(position)
        cols.extend(section_time)
        cols.extend(margin_behind)
        
        cols.append(date)
        cols.append(match)
        
        output_list.append(cols)
 
    return output_list

In [22]:
filepath = "section_time_git_2.csv"

YEAR_START = 2021
YEAR_END = 2021

MONTH_START = 1
MONTH_END = 12


import time


# standard csv saving
url_front = "https://racing.hkjc.com/racing/information/english/Racing/DisplaySectionalTime.aspx?RaceDate="

import csv
with open(filepath, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile)
    mywriter.writerow(["finishing_order", "horse_no", "horse", "time",
                       
                       "section_time_1", "section_time_2", "section_time_3", 
                       "section_time_4", "section_time_5", "section_time_6",
                       
                       "margin_behind_1", "margin_behind_2", "margin_behind_3", 
                       "margin_behind_4", "margin_behind_5", "margin_behind_6", 
                       
                       "date", "match"])
    
    #Copy the loop from above and incorporate the csv-saving code
    for year in range(YEAR_START, YEAR_END+1):
        for month in range(MONTH_START, MONTH_END+1):
            start_time = time.time()
            for day in range(1,32):
                month_2d = '{:02d}'.format(month)
                day_2d = '{:02d}'.format(day)
                date = day_2d + "/" + month_2d + "/" + str(year)
                url = url_front + date
                
                # check if it is a race day before proceed
                check = table(url, date)
                
                if len(check) > 0:
                    for match in range (1, 13):
                        time.sleep(3)
                        
                        #Print the URL so we know the progress so far
                        #print("Trying:",url)
                        
                        url = url_front + date + "&RaceNo=" + str(match) #01/01/2010&RaceNo=1

                        #Call our function to fetch and process data given the URL
                        content = table(url, date, match)  # ***** what really write into csv *****
                        content = np.array(content)

                        if len(content) > 0:
                            mywriter.writerows(content)

            print(year, month, "done", (time.time() - start_time)/60)

driver.close()

Message: 
 01/02/2021 1
Message: 
 02/02/2021 1
Message: 
 03/02/2021 10
Message: 
 03/02/2021 11
Message: 
 03/02/2021 12
Message: 
 04/02/2021 1
Message: 
 05/02/2021 1
Message: 
 06/02/2021 11
Message: 
 06/02/2021 12
Message: 
 07/02/2021 1
Message: 
 08/02/2021 1
Message: 
 09/02/2021 1
Message: 
 10/02/2021 10
Message: 
 10/02/2021 11
Message: 
 10/02/2021 12
Message: 
 11/02/2021 1
Message: 
 12/02/2021 1
Message: 
 13/02/2021 1
Message: 
 14/02/2021 12
Message: 
 15/02/2021 1
Message: 
 16/02/2021 1
Message: 
 17/02/2021 10
Message: 
 17/02/2021 11
Message: 
 17/02/2021 12
Message: 
 18/02/2021 1
Message: 
 19/02/2021 1
Message: 
 20/02/2021 1
Message: 
 21/02/2021 11
Message: 
 21/02/2021 12
Message: 
 22/02/2021 1
Message: 
 23/02/2021 1
Message: 
 24/02/2021 10
Message: 
 24/02/2021 11
Message: 
 24/02/2021 12
Message: 
 25/02/2021 1
Message: 
 26/02/2021 1
Message: 
 27/02/2021 1
Message: 
 28/02/2021 11
Message: 
 28/02/2021 12
Message: 
 29/02/2021 1
Message: 
 30/02/2021